In [1]:
import numpy as np 
import pandas as pd 
import datetime as dt

In [2]:
data = pd.read_csv('DatosBolivia.csv', sep=';', decimal=',')

In [3]:
data

,fecha1,fecha2,hora,BOLIVIA
0,FECHA,FECHA2,Hora,98011.0
1,1/1/2010 00:00,31/12/2009 00:00,00:00,0.0
2,1/1/2010 01:00,1/1/2010 00:00,01:00,0.0
3,1/1/2010 02:00,1/1/2010 00:00,02:00,0.0
4,1/1/2010 03:00,1/1/2010 00:00,03:00,0.0
...,...,...,...,...
86277,4/11/2019 20:00,4/11/2019 00:00,20:00,NaN
86278,4/11/2019 21:00,4/11/2019 00:00,21:00,NaN
86279,4/11/2019 22:00,4/11/2019 00:00,22:00,NaN
86280,4/11/2019 23:00,4/11/2019 00:00,23:00,NaN


In [58]:
data = data[1:]

In [59]:
data.loc['fecha1'] = pd.to_datetime(data.fecha1)

In [60]:
data = data.drop('fecha2',axis=1).drop('hora',axis=1)

In [61]:
data

,fecha1,BOLIVIA
1,1/1/2010 00:00,0.0
2,1/1/2010 01:00,0.0
3,1/1/2010 02:00,0.0
4,1/1/2010 03:00,0.0
5,1/1/2010 04:00,0.0
...,...,...
86278,4/11/2019 21:00,NaN
86279,4/11/2019 22:00,NaN
86280,4/11/2019 23:00,NaN
86281,5/11/2019 00:00,NaN


In [62]:
data.index = pd.DatetimeIndex(data['fecha1'])

In [63]:
#Elimino duplicados
data = data.drop_duplicates('fecha1')

In [64]:
#lista de todo el rango de tiempo
all_days = pd.date_range(data.index.min(), data.index.max(),freq='h')

In [65]:
all_days

DatetimeIndex(['2010-01-01 00:00:00', '2010-01-01 01:00:00',
               '2010-01-01 02:00:00', '2010-01-01 03:00:00',
               '2010-01-01 04:00:00', '2010-01-01 05:00:00',
               '2010-01-01 06:00:00', '2010-01-01 07:00:00',
               '2010-01-01 08:00:00', '2010-01-01 09:00:00',
               ...
               '2019-12-10 14:00:00', '2019-12-10 15:00:00',
               '2019-12-10 16:00:00', '2019-12-10 17:00:00',
               '2019-12-10 18:00:00', '2019-12-10 19:00:00',
               '2019-12-10 20:00:00', '2019-12-10 21:00:00',
               '2019-12-10 22:00:00', '2019-12-10 23:00:00'],
              dtype='datetime64[ns]', length=87144, freq='H')

In [66]:
#compruebo que el rango de todos los días sea igual o mayor a la cantidad de datos que tengo y completo la matriz
if len(all_days) >=  data.shape[0] :
    Complete_Data =data.reindex(all_days)

In [67]:
nan_rows = Complete_Data[Complete_Data.isnull().T.any().T]

In [68]:
#Conteo mensual de datos faltantes
faltantes_mensuales = nan_rows.isnull().groupby(pd.Grouper(freq='1M')).sum().astype(int)
faltantes_mensuales.index = faltantes_mensuales.index.strftime('%B %Y')
faltantes_mensuales['NaN'] = faltantes_mensuales['fecha1']
faltantes_mensuales = faltantes_mensuales['NaN']
faltantes_mensuales

January 2010        0
February 2010       0
March 2010          0
April 2010          0
May 2010            0
                 ... 
August 2019        48
September 2019     48
October 2019       48
November 2019     480
December 2019       0
Name: NaN, Length: 120, dtype: int32

In [69]:
#cantidad de fechas faltantes
faltantes_tot = nan_rows.shape[0]
faltantes_tot

6435

In [70]:
#identificamos cuales entradas no están en el formato correcto
x= []
for i in range(Complete_Data['BOLIVIA'].shape[0]):
    if isinstance(Complete_Data.loc[:,'BOLIVIA'].iloc[i], float) == False:
        x.append(i)

In [71]:
#convertimos a float las columnas según el error al importar los datos
y = len(x)
for i in range(0,y):
    Complete_Data.loc[:,'BOLIVIA'].iloc[x[i]] = float(Complete_Data.loc[:,'BOLIVIA'].iloc[x[i]])

In [72]:
del x
del y

In [75]:
def Estacion(var):
    no_representativos = []
    anomalos = []
    vacio = []
    fechas = []
    tabla = []
    new = var
    start = []
    end = []
    j = -1
    idx = len(new.index)
    headers = list(new.T.index)
    dates = new.index
    t_min=60
    for i in dates:
        j = j+1
        if new.loc[:,'BOLIVIA'].loc[i] < 0 or new.loc[:,'BOLIVIA'].loc[i] >100:
            fechas.append(str(i))
            if j < idx - 1 :
                if j==0:
                    if i + dt.timedelta(minutes=t_min) == dates[j+1] :
                        if new.loc[:,'BOLIVIA'].iloc[j+1] < 0 or new.loc[:,'BOLIVIA'].iloc[j+1] >100 :
                            start.append(i)
                    else: 
                        string = str(i) + ' no representativo'
                        print(string)
                        no_representativos.append(str(i))
                        del string
                elif i + dt.timedelta(minutes=t_min) == dates[j+1] :
                    if new.loc[:,'BOLIVIA'].iloc[j+1] < 0 or new.loc[:,'BOLIVIA'].iloc[j+1] >100 :
                        start.append(i)
                    elif i - dt.timedelta(minutes=t_min) == dates[j-1]:
                        if new.loc[:,'BOLIVIA'].iloc[j-1] < 0 or new.loc[:,'BOLIVIA'].iloc[j-1] >100:
                            end.append(i)
                            string = 'Desde ' + str(start[0]) + ' hasta ' + str(end[0]) + ' no representativos'
                            no_representativos.append(str(start[0]) + ' a ' + str(end[0]))
                            print(string)
                            start = []
                            end = []
                            del string
                    else:
                        string = str(i) + ' no representativo'
                        no_representativos.append(str(i))
                        print(string)
                        del string
                elif  i - dt.timedelta(minutes=t_min) == dates[j-1]: 
                    if new.loc[:,'BOLIVIA'].iloc[j-1] < 0 or new.loc[:,'BOLIVIA'].iloc[j-1] >100 :
                        end.append(i)
                        string = 'Desde ' + str(start[0]) + ' hasta ' + str(end[0]) + ' no representativos'
                        no_representativos.append(str(start[0]) + ' a ' + str(end[0]))
                        print(string)
                        del string
                        start = []
                        end = []
                else:
                    string = str(i) + ' no representativo'
                    no_representativos.append(str(i))
                    print(string)
                    del string
            elif j == idx -1:
                if i - dt.timedelta(minutes=t_min) != dates[j-1]:
                    string = str(i) + ' no representativo'
                    no_representativos.append(str(i))
                    print(string)
                    del string
                else:
                    end.append(i)
                    string = 'Desde ' + str(start[0]) + ' hasta ' + str(end[0]) + ' no representativos'
                    no_representativos.append(str(start[0]) + ' a ' + str(end[0]))
                    print(string)
                    start = []
                    end = []
                    del string
        elif ('BOLIVIA' in list(headers)) == True :
            fechas.append(str(i))
            if j < idx - 1 :
                if pd.isna(new.loc[:,'BOLIVIA'].loc[i]) == True:
                    if j==0:
                        if i + dt.timedelta(minutes=t_min) == dates[j+1]:
                            if pd.isna(new.loc[:,'BOLIVIA'].iloc[j+1]) == True :
                                start.append(i)
                        else:
                            string = str(i) + ' Sin dato'
                            print(string)
                            vacio.append(str(i))
                            del string
                    elif i + dt.timedelta(minutes=t_min) == dates[j+1]:
                        if pd.isna(new.loc[:,'BOLIVIA'].iloc[j+1]) == True :
                            start.append(i)
                        elif i - dt.timedelta(minutes=t_min) == dates[j-1]:
                            if pd.isna(new.loc[:,'BOLIVIA'].iloc[j-1]) == True:
                                end.append(i)
                                string = 'Desde ' + str(start[0]) + ' hasta ' + str(end[0]) + ' sin datos'
                                vacio.append(str(start[0]) + ' a ' + str(end[0]))
                                print(string)
                                start = []
                                end = []
                                del string
                        else:
                            string = str(i) + ' sin dato'
                            print(string)
                            vacio.append(str(i))
                            del string
                    elif  i - dt.timedelta(minutes=t_min) == dates[j-1] : 
                        if pd.isna(new.loc[:,'BOLIVIA'].iloc[j-1]) == True :
                            end.append(i)
                            string = 'Desde ' + str(start[0]) + ' hasta ' + str(end[0]) + ' sin datos'
                            vacio.append(str(start[0]) + ' a ' + str(end[0]))
                            print(string)
                            start = []
                            end = []
                            del string
                    else:
                        string = str(i) + ' sin dato'
                        print(str(i) + ' sin dato')
                        vacio.append(str(i))
                        del string
    tabla.append(fechas)
    tabla.append(anomalos)
    tabla.append(no_representativos)
    tabla.append(vacio)
    return tabla

In [76]:
bol = Estacion(Complete_Data)

Desde 2010-01-07 00:00:00 hasta 2010-01-07 10:00:00 sin datos
Desde 2010-06-20 02:00:00 hasta 2010-06-30 23:00:00 sin datos
Desde 2011-01-07 01:00:00 hasta 2011-01-08 23:00:00 sin datos
Desde 2011-01-09 01:00:00 hasta 2011-01-09 23:00:00 sin datos
Desde 2011-02-07 00:00:00 hasta 2011-02-09 23:00:00 sin datos
Desde 2011-03-07 00:00:00 hasta 2011-03-09 23:00:00 sin datos
Desde 2011-04-07 00:00:00 hasta 2011-04-09 23:00:00 sin datos
Desde 2011-05-07 00:00:00 hasta 2011-05-09 23:00:00 sin datos
Desde 2011-06-05 07:00:00 hasta 2011-06-05 08:00:00 sin datos
Desde 2011-06-07 00:00:00 hasta 2011-06-09 23:00:00 sin datos
Desde 2011-07-07 00:00:00 hasta 2011-07-09 23:00:00 sin datos
Desde 2011-07-13 00:00:00 hasta 2011-07-31 23:00:00 sin datos
Desde 2011-08-07 00:00:00 hasta 2011-08-09 23:00:00 sin datos
Desde 2011-08-13 00:00:00 hasta 2011-08-31 23:00:00 sin datos
Desde 2011-09-07 00:00:00 hasta 2011-09-09 23:00:00 sin datos
Desde 2011-09-13 00:00:00 hasta 2011-09-30 23:00:00 sin datos
Desde 20

In [ ]:
est=Estacion(Complete_Data)
est_anom = pd.DataFrame([est[1]], index = ['Anómalos'])
est_norep = pd.DataFrame([est[2]], index = ['No_Representativos'])
est_vaci = pd.DataFrame([est[3]], index = ['Vacios'])

est_anom.T.to_csv('est' +'_'+'anomalos', index = False)
est_norep.T.to_csv('est' +'_'+'no_represent', index = False)
est_vaci.T.to_csv('est' +'_'+'vacios', index = False)